<a href="https://colab.research.google.com/github/Stefmeff/ML_ImageUpscaling/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone the repo into the content folder:**

In [ ]:
%%bash
REPO_DIR="/content/ML_ImageUpscaling"

if [ -d "$REPO_DIR/.git" ]; then
  cd "$REPO_DIR" && git pull
else
  git clone https://github.com/Stefmeff/ML_ImageUpscaling.git "$REPO_DIR"
fi

cd "$REPO_DIR"


Updating 25acade..8ecb833
Fast-forward
 data/images_train/iamge5.jpg | Bin 0 -> 23907 bytes
 data/images_train/image2.png | Bin 0 -> 74388 bytes
 data/images_train/image3.jpg | Bin 0 -> 20973 bytes
 data/images_train/image4.png | Bin 0 -> 86487 bytes
 4 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 data/images_train/iamge5.jpg
 create mode 100644 data/images_train/image2.png
 create mode 100644 data/images_train/image3.jpg
 create mode 100644 data/images_train/image4.png


From https://github.com/Stefmeff/ML_ImageUpscaling
   25acade..8ecb833  main       -> origin/main


**Download Dependencies:**

In [ ]:
!pip install torch numpy pillow h5py tqdm!pip install torch numpy pillow h5py tqdm

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/113.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/113.7 MB 4.2 MB/s eta 0:00:28
    --------------------------------------- 2.1/113.7 MB 5.6 MB/s eta 0:00:20
   - -------------------------------------- 3.7/113.7 MB 6.1 MB/s eta 0:00:19
   - -------------------------------------- 5.2/113.7 MB 6.4 MB/s eta 0:00:18
   -- ------------------------------------- 7.1/113.7 MB 7.0 MB/s eta 0:00:16
   --- ------------------------------------ 8.9/113.7 MB 7.4 MB/s eta 0:00:15
   --- ------------------------------------ 11.0/113.7 MB 7.6 MB/s eta 0:00:14
   ---- ----------------------------------- 12.8/113.7 MB 7.7 MB/s eta 0:00:14
   ----- ---------------------------------- 14.7/113.7 MB 7.8 MB/s eta 0:00:13
   ----- ---------------------------------- 16.5/113.7 MB 7.9 MB/s eta 0:00:13
   ------ ----------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8 [torch]
   ----------------------------------- ---- 7/8

In [ ]:
%cd /content/ML_ImageUpscaling

**Download Dataset:**

Note: Before you execute this, you need to set new variables KAGGLE_USERNAME and KAGGLE_KEY in collab **secrets**. You can obtain an kaggle api key from your kaggle account.

In [ ]:
# Configure Kaggle API credentials from environment variables
!pip install kaggle
from google.colab import userdata
import os
import shutil

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

DATASET="soumikrakshit/div2k-high-resolution-images"
DATA_DIR="/content/ML_ImageUpscaling/data"

# Download the dataset
!kaggle datasets download -d $DATASET -p $DATA_DIR
!unzip -q $DATA_DIR/div2k-high-resolution-images.zip -d $DATA_DIR

# Rename and reformat the folders for training, testing, and evaluation:
!mv /content/ML_ImageUpscaling/data/DIV2K_train_HR/DIV2K_train_HR /content/ML_ImageUpscaling/data/images_train
!mv /content/ML_ImageUpscaling/data/DIV2K_valid_HR/DIV2K_valid_HR /content/ML_ImageUpscaling/data/images_test
!mkdir /content/ML_ImageUpscaling/data/images_eval

!rmdir /content/ML_ImageUpscaling/data/DIV2K_train_HR
!rmdir /content/ML_ImageUpscaling/data/DIV2K_valid_HR

train_dir = "/content/ML_ImageUpscaling/data/images_train"
eval_dir = "/content/ML_ImageUpscaling/data/images_eval"


images = sorted([f for f in os.listdir(train_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
# Move last 100 images for evaluation
for img_name in images[-100:]:
    src = os.path.join(train_dir, img_name)
    dst = os.path.join(eval_dir, img_name)
    shutil.move(src, dst)


**Prepare Dataset:**

In [ ]:
# Specify number of images for training and evaluation
num_images = 300
num_train = int(num_images * 0.8)
num_eval = int(num_images * 0.2)


# Prepare training and evaluation datasets
!python src/prepare.py --images-dir data/images_train --num-images $num_train --output-path data/train_x3.h5 --scale 3 --patch-size 8
!python src/prepare.py --images-dir data/images_eval --num-images $num_eval --output-path data/eval_x3.h5 --scale 3 --eval

# Check files created
!ls -lh data/train_x3.h5 data/eval_x3.h5

**Train Model:**

In [ ]:
#Check if GPU is available
import torch
torch.cuda.is_available()

In [91]:
!python src/train.py --train-file data/train_x3.h5 --eval-file data/eval_x3.h5 --outputs-dir outputs --scale 3 --num-epochs 10 --num-workers 0

epoch: 0/9:   0% 0/880 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
epoch: 0/9: 100% 880/880 [00:01<00:00, 505.46it/s, loss=0.070437]
eval psnr: 7.83
epoch: 1/9: 100% 880/880 [00:01<00:00, 646.75it/s, loss=0.009499]
eval psnr: 6.89
epoch: 2/9: 100% 880/880 [00:01<00:00, 634.99it/s, loss=0.002361]
eval psnr: 6.68
epoch: 3/9: 100% 880/880 [00:01<00:00, 628.90it/s, loss=0.001215]
eval psnr: 6.75
epoch: 4/9: 100% 880/880 [00:01<00:00, 647.49it/s, loss=0.000818]
eval psnr: 6.79
epoch: 5/9: 100% 880/880 [00:01<00:00, 651.72it/s, loss=0.000656]
eval psnr: 6.81
epoch: 6/9: 100% 880/880 [00:01<00:00, 454.73it/s, loss=0.000593]
eval psnr: 6.86
epoch: 7/9: 100% 880/880 [00:01<00:00, 645.21it/s, loss=0.000537]
eval psnr: 6.93
epoch: 8/9: 100% 880/880 [00:01<00:00, 654.92it/s, loss=0.000464]
eval psnr: 7.01
e

**Test Model:**

In [ ]:
!python src/test.py --weights-file outputs/x3/best.pth --image-file data/images_test/image4.png --scale 3

PSNR: 7.83
